# AML Feature Store - Análise e Demonstração

Este notebook demonstra como usar o Feature Store para análise de dados históricos e treinamento de modelos de ML para detecção de lavagem de dinheiro.

## Objetivos
1. Explorar dados históricos usando o Feast Offline Store
2. Analisar padrões de transações suspeitas
3. Criar datasets de treinamento consistentes
4. Treinar um modelo simples de detecção de fraude
5. Demonstrar a consistência entre ambiente offline e online


In [ ]:
# Imports necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configurar visualizações
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("📦 Bibliotecas importadas com sucesso!")


In [ ]:
# Imports específicos do projeto
import sys
sys.path.append('../')

from feast import FeatureStore
import redis
import json
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.feature_selection import SelectKBest, f_classif, RFE
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

# Inicializar Feature Store
try:
    store = FeatureStore(repo_path="../feature_repo")
    print("🎯 Feature Store inicializado!")
except Exception as e:
    print(f"⚠️ Erro ao inicializar Feature Store: {e}")
    store = None

# Conectar ao Redis (Online Store)
try:
    redis_client = redis.Redis(host='localhost', port=6379, decode_responses=True)
    redis_client.ping()
    print("🔗 Conectado ao Redis!")
except Exception as e:
    print(f"⚠️ Erro ao conectar ao Redis: {e}")
    redis_client = None

print("📦 Bibliotecas avançadas importadas com sucesso!")


## 1. Geração e Análise Avançada de Dados


In [ ]:
# Gerar dados sintéticos avançados se não existirem
import os
if not os.path.exists('../offline_data/transactions.parquet'):
    print("📝 Gerando dados sintéticos avançados...")
    exec(open('../offline_data/generate_sample_data.py').read())
else:
    print("📁 Dados já existem, carregando...")

# Carregar dados históricos
df = pd.read_parquet('../offline_data/transactions.parquet')
print(f"📈 Dados carregados: {len(df):,} transações")
print(f"📅 Período: {df['event_timestamp'].min()} até {df['event_timestamp'].max()}")
print(f"👥 Clientes únicos: {df['customer_id'].nunique():,}")
print(f"🏪 Estabelecimentos únicos: {df['merchant_id'].nunique():,}")

# Análise de distribuição temporal
df['hour'] = pd.to_datetime(df['event_timestamp']).dt.hour
df['day_of_week'] = pd.to_datetime(df['event_timestamp']).dt.dayofweek
df['is_business_hours'] = df['hour'].between(9, 17)

print(f"\n📊 Estatísticas básicas:")
print(f"💰 Valor médio: R$ {df['amount'].mean():.2f}")
print(f"📊 Mediana: R$ {df['amount'].median():.2f}")
print(f"🚨 Taxa de suspeitas: {df['is_suspicious'].mean()*100:.1f}%")
print(f"🌙 Transações noturnas: {(df['hour'] < 6).sum() + (df['hour'] > 22).sum()} ({((df['hour'] < 6).sum() + (df['hour'] > 22).sum())/len(df)*100:.1f}%)")

df.head()


## 2. Engenharia de Features Avançada


In [ ]:
# Função avançada para calcular features temporais
def calculate_advanced_features(df, customer_id, timestamp, windows):
    """Calcula features avançadas de janela temporal com padrões comportamentais"""
    
    # Filtrar transações do cliente até o timestamp
    customer_txns = df[
        (df['customer_id'] == customer_id) & 
        (df['event_timestamp'] <= timestamp)
    ].copy()
    
    features = {}
    
    for window_name, window_minutes in windows.items():
        # Calcular cutoff time
        cutoff_time = timestamp - timedelta(minutes=window_minutes)
        
        # Transações na janela
        window_txns = customer_txns[customer_txns['event_timestamp'] > cutoff_time]
        
        if len(window_txns) == 0:
            # Features básicas zeradas
            features.update({
                f'txn_count_{window_name}': 0,
                f'txn_amount_sum_{window_name}': 0.0,
                f'avg_txn_amount_{window_name}': 0.0,
                f'max_txn_amount_{window_name}': 0.0,
                f'min_txn_amount_{window_name}': 0.0,
                f'std_txn_amount_{window_name}': 0.0,
                f'unique_merchants_{window_name}': 0,
                f'unique_ips_{window_name}': 0,
                f'velocity_score_{window_name}': 0.0,
                f'night_txn_ratio_{window_name}': 0.0,
                f'weekend_txn_ratio_{window_name}': 0.0,
                f'business_hours_ratio_{window_name}': 0.0,
                f'amount_concentration_gini_{window_name}': 0.0,
                f'merchant_concentration_hhi_{window_name}': 0.0
            })
            continue
        
        # Features básicas
        amounts = window_txns['amount'].values
        features[f'txn_count_{window_name}'] = len(window_txns)
        features[f'txn_amount_sum_{window_name}'] = amounts.sum()
        features[f'avg_txn_amount_{window_name}'] = amounts.mean()
        features[f'max_txn_amount_{window_name}'] = amounts.max()
        features[f'min_txn_amount_{window_name}'] = amounts.min()
        features[f'std_txn_amount_{window_name}'] = amounts.std() if len(amounts) > 1 else 0.0
        
        # Features de diversidade
        features[f'unique_merchants_{window_name}'] = window_txns['merchant_id'].nunique()
        features[f'unique_ips_{window_name}'] = window_txns['ip_address'].nunique()
        
        # Features temporais
        features[f'velocity_score_{window_name}'] = len(window_txns) / (window_minutes / 60.0)  # txn por hora
        
        # Ratios comportamentais
        night_txns = window_txns[(window_txns['hour'] < 6) | (window_txns['hour'] > 22)]
        features[f'night_txn_ratio_{window_name}'] = len(night_txns) / len(window_txns)
        
        weekend_txns = window_txns[window_txns['day_of_week'] >= 5]
        features[f'weekend_txn_ratio_{window_name}'] = len(weekend_txns) / len(window_txns)
        
        business_txns = window_txns[window_txns['is_business_hours']]
        features[f'business_hours_ratio_{window_name}'] = len(business_txns) / len(window_txns)
        
        # Features de concentração (indicadores de anomalia)
        # Gini coefficient para distribuição de valores
        if len(amounts) > 1:
            sorted_amounts = np.sort(amounts)
            n = len(sorted_amounts)
            cumsum = np.cumsum(sorted_amounts)
            gini = (n + 1 - 2 * np.sum(cumsum) / cumsum[-1]) / n
            features[f'amount_concentration_gini_{window_name}'] = gini
        else:
            features[f'amount_concentration_gini_{window_name}'] = 0.0
        
        # Herfindahl-Hirschman Index para concentração de estabelecimentos
        merchant_counts = window_txns['merchant_id'].value_counts()
        merchant_shares = merchant_counts / merchant_counts.sum()
        hhi = (merchant_shares ** 2).sum()
        features[f'merchant_concentration_hhi_{window_name}'] = hhi
        
        # Features de padrão sequencial
        if len(window_txns) > 1:
            # Tempo médio entre transações
            time_diffs = window_txns['event_timestamp'].diff().dt.total_seconds().dropna()
            if len(time_diffs) > 0:
                features[f'avg_time_between_txns_{window_name}'] = time_diffs.mean()
                features[f'std_time_between_txns_{window_name}'] = time_diffs.std()
            else:
                features[f'avg_time_between_txns_{window_name}'] = 0.0
                features[f'std_time_between_txns_{window_name}'] = 0.0
        else:
            features[f'avg_time_between_txns_{window_name}'] = 0.0
            features[f'std_time_between_txns_{window_name}'] = 0.0
    
    return features

# Definir janelas temporais mais granulares
time_windows = {
    '1m': 1,
    '5m': 5, 
    '15m': 15,
    '1h': 60,
    '6h': 360,
    '24h': 1440
}

print("🔧 Função de engenharia de features avançada criada!")
print(f"📊 Janelas temporais: {list(time_windows.keys())}")
print(f"🎯 Features por janela: ~15 features × {len(time_windows)} janelas = ~{15 * len(time_windows)} features")


## 3. Modelos Avançados de Machine Learning


In [ ]:
# Criar dataset avançado com features engineered
print("🔄 Criando dataset avançado com features engineered...")
print("⚠️ Isso pode levar alguns minutos para processar...")

# Usar uma amostra maior para modelos mais robustos
sample_size = 10000
df_sample = df.sample(n=min(sample_size, len(df)), random_state=42).sort_values('event_timestamp')

advanced_training_data = []

for idx, row in df_sample.iterrows():
    if idx % 2000 == 0:
        print(f"   Processando transação {idx}/{len(df_sample)}...")
    
    # Calcular features avançadas para esta transação
    features = calculate_advanced_features(
        df, row['customer_id'], row['event_timestamp'], time_windows
    )
    
    # Adicionar informações da transação atual
    record = {
        'transaction_id': row['transaction_id'],
        'customer_id': row['customer_id'],
        'merchant_id': row['merchant_id'],
        'amount': row['amount'],
        'hour': row['hour'],
        'day_of_week': row['day_of_week'],
        'is_weekend': row['is_weekend'],
        'is_business_hours': row['is_business_hours'],
        'event_timestamp': row['event_timestamp'],
        'is_suspicious': row['is_suspicious'],  # Target variable
        **features
    }
    
    advanced_training_data.append(record)

# Converter para DataFrame
training_df_advanced = pd.DataFrame(advanced_training_data)

print(f"✅ Dataset avançado criado: {len(training_df_advanced)} amostras")
print(f"📊 Total de features: {len(training_df_advanced.columns) - 6}")  # Excluir colunas não-feature

# Identificar colunas de features
feature_columns = [col for col in training_df_advanced.columns 
                  if col not in ['transaction_id', 'customer_id', 'merchant_id', 
                               'event_timestamp', 'is_suspicious']]

print(f"🎯 Features para modelagem: {len(feature_columns)}")

# Mostrar algumas estatísticas
print(f"\n📈 Distribuição do target:")
print(training_df_advanced['is_suspicious'].value_counts())
print(f"Taxa de desbalanceamento: {training_df_advanced['is_suspicious'].value_counts()[0] / training_df_advanced['is_suspicious'].value_counts()[1]:.1f}:1")

training_df_advanced.head()


In [ ]:
# Preparar dados para modelagem avançada
print("🔄 Preparando dados para modelagem...")

# Selecionar features e target
X = training_df_advanced[feature_columns].fillna(0)  # Preencher NaN com 0
y = training_df_advanced['is_suspicious']

print(f"📊 Shape dos dados: {X.shape}")
print(f"🎯 Distribuição do target: {y.value_counts().to_dict()}")

# Análise de correlação avançada
correlation_matrix = X.corrwith(y).sort_values(key=abs, ascending=False)
print(f"\n🔍 Top 10 features mais correlacionadas:")
for feature, corr in correlation_matrix.head(10).items():
    print(f"  {feature}: {corr:.4f}")

# Visualizar correlações
plt.figure(figsize=(12, 8))
top_features = correlation_matrix.head(15)
colors = ['red' if x > 0 else 'blue' for x in top_features.values]
bars = plt.barh(range(len(top_features)), top_features.values, color=colors, alpha=0.7)
plt.yticks(range(len(top_features)), top_features.index, fontsize=10)
plt.xlabel('Correlação com Target (is_suspicious)')
plt.title('Top 15 Features Mais Correlacionadas')
plt.grid(axis='x', alpha=0.3)

for i, (bar, value) in enumerate(zip(bars, top_features.values)):
    plt.text(value + 0.01 if value > 0 else value - 0.01, i, f'{value:.3f}', 
             va='center', ha='left' if value > 0 else 'right', fontsize=8)

plt.tight_layout()
plt.show()


In [ ]:
# Treinamento de múltiplos modelos avançados
print("🤖 Treinando múltiplos modelos de ML...")

# Split dos dados
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"📊 Treino: {X_train.shape[0]} amostras")
print(f"📊 Teste: {X_test.shape[0]} amostras")

# Definir modelos para comparação
models = {
    'Random Forest': RandomForestClassifier(
        n_estimators=100, max_depth=10, min_samples_split=5,
        random_state=42, class_weight='balanced'
    ),
    'XGBoost': xgb.XGBClassifier(
        n_estimators=100, max_depth=6, learning_rate=0.1,
        random_state=42, scale_pos_weight=10  # Para desbalanceamento
    ),
    'Gradient Boosting': GradientBoostingClassifier(
        n_estimators=100, max_depth=6, learning_rate=0.1,
        random_state=42
    ),
    'Logistic Regression': LogisticRegression(
        random_state=42, class_weight='balanced', max_iter=1000
    )
}

# Treinar e avaliar modelos
results = {}
trained_models = {}

for name, model in models.items():
    print(f"\n🔄 Treinando {name}...")
    
    # Treinar modelo
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    # Fazer predições
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Calcular métricas
    auc_score = roc_auc_score(y_test, y_pred_proba)
    
    results[name] = {
        'auc_score': auc_score,
        'training_time': training_time,
        'predictions': y_pred,
        'probabilities': y_pred_proba
    }
    
    trained_models[name] = model
    
    print(f"  ✅ AUC: {auc_score:.4f} | Tempo: {training_time:.2f}s")

# Mostrar comparação de modelos
print(f"\n📊 COMPARAÇÃO DE MODELOS:")
print("=" * 50)
for name, metrics in results.items():
    print(f"{name:20} | AUC: {metrics['auc_score']:.4f} | Tempo: {metrics['training_time']:.2f}s")

# Identificar melhor modelo
best_model_name = max(results.keys(), key=lambda x: results[x]['auc_score'])
best_model = trained_models[best_model_name]
print(f"\n🏆 Melhor modelo: {best_model_name} (AUC: {results[best_model_name]['auc_score']:.4f})")


## 4. Análise Detalhada do Melhor Modelo


In [ ]:
# Análise detalhada do melhor modelo
print(f"🔍 Analisando modelo: {best_model_name}")

# Obter predições do melhor modelo
best_predictions = results[best_model_name]['predictions']
best_probabilities = results[best_model_name]['probabilities']

# Visualizações avançadas
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Confusion Matrix
cm = confusion_matrix(y_test, best_predictions)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0,0])
axes[0,0].set_title(f'Matriz de Confusão - {best_model_name}')
axes[0,0].set_xlabel('Predito')
axes[0,0].set_ylabel('Real')

# 2. ROC Curve
fpr, tpr, _ = roc_curve(y_test, best_probabilities)
auc = results[best_model_name]['auc_score']
axes[0,1].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC (AUC = {auc:.3f})')
axes[0,1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes[0,1].set_xlim([0.0, 1.0])
axes[0,1].set_ylim([0.0, 1.05])
axes[0,1].set_xlabel('Taxa de Falsos Positivos')
axes[0,1].set_ylabel('Taxa de Verdadeiros Positivos')
axes[0,1].set_title('Curva ROC')
axes[0,1].legend(loc="lower right")
axes[0,1].grid(alpha=0.3)

# 3. Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_test, best_probabilities)
axes[1,0].plot(recall, precision, color='purple', lw=2)
axes[1,0].set_xlabel('Recall')
axes[1,0].set_ylabel('Precision')
axes[1,0].set_title('Curva Precision-Recall')
axes[1,0].grid(alpha=0.3)

# 4. Feature Importance (se disponível)
if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': feature_columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False).head(15)
    
    axes[1,1].barh(range(len(feature_importance)), feature_importance['importance'], color='lightgreen')
    axes[1,1].set_yticks(range(len(feature_importance)))
    axes[1,1].set_yticklabels(feature_importance['feature'], fontsize=8)
    axes[1,1].set_xlabel('Importância')
    axes[1,1].set_title('Top 15 Features Mais Importantes')
    axes[1,1].grid(axis='x', alpha=0.3)
else:
    axes[1,1].text(0.5, 0.5, 'Feature importance\nnão disponível\npara este modelo', 
                   ha='center', va='center', transform=axes[1,1].transAxes)
    axes[1,1].set_title('Feature Importance')

plt.tight_layout()
plt.show()

# Relatório detalhado
print(f"\n📊 RELATÓRIO DETALHADO - {best_model_name}")
print("=" * 60)
print(classification_report(y_test, best_predictions, target_names=['Normal', 'Suspeita']))

# Análise de threshold
thresholds = np.arange(0.1, 1.0, 0.1)
threshold_metrics = []

for threshold in thresholds:
    y_pred_thresh = (best_probabilities >= threshold).astype(int)
    
    if len(np.unique(y_pred_thresh)) > 1:  # Evitar divisão por zero
        precision = precision_score(y_test, y_pred_thresh)
        recall = recall_score(y_test, y_pred_thresh)
        f1 = f1_score(y_test, y_pred_thresh)
    else:
        precision = recall = f1 = 0
    
    threshold_metrics.append({
        'threshold': threshold,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    })

threshold_df = pd.DataFrame(threshold_metrics)
print(f"\n🎯 ANÁLISE DE THRESHOLD:")
print(threshold_df.round(3))
